# Data

This notebook downloads data for various cities to the `data\` folder.

In [53]:
import os
import googlemaps
import json
import time

In [83]:
import pandas as pd
import flatdict

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
gmaps = googlemaps.Client(key=os.environ["GOOGLE_GEOCODING_API_KEY"])

In [119]:
def data_list_to_df(data):
    
    result_list = list()
    for d in data:
        result_list += [flatdict.FlatDict(r, delimiter=".") for r in d["results"]]
        
    df = pd.DataFrame(result_list)
    return df

In [120]:
def get_data(keyword: str, location=(49.1426929, 9.210879), radius=5000):
    """Given a keyword and a (lat, lon) location get a dataframe of places within the radius of Xmeters.
    """
    res_list = list()
    first_res = gmaps.places_nearby(keyword=keyword, location=location, radius=radius)
    res_list.append(first_res)
    
    next_page_token = res_list[0].get("next_page_token", None)
    
    while next_page_token:
        time.sleep(3)
        next_result = gmaps.places_nearby(page_token=next_page_token)
        next_page_token =  next_result.get("next_page_token", None)
        res_list.append(next_result)
        
    df = data_list_to_df(res_list)
    return df

In [116]:
cities = {"Heilbronn": (49.1426929, 9.210879),
          "Stuttgart": (48.78232, 9.17702),
          "Frankfurt": (50.107345, 8.664046),
          "Marzahn": (52.544260, 13.564032),
         "Berlin-Neukölln": (52.481389, 13.435278),
         "Köln-Kalk": (50.935833, 7.018611),
         "Hamburg-Altona": (53.55, 9.933333),
         "Frankfurt-Hbf": (50.107443, 8.664446)}

In [117]:
for food in ["Döner", "Pizza"]:
    for k, v in cities.items():
        res_path = os.path.join("data", f"{food.lower()}_{k.lower()}.csv")
        if os.path.exists(res_path):
            continue
        res_df = get_data(food, v)
        res_df.to_csv(res_path)